In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
import os
import keras
from keras import backend as K
print(tf.__version__)
%cd /content/drive/My\ Drive/Colab\ Notebooks/Rootee
#tf.config.experimental_run_functions_eagerly(True)

1.15.0
/content/drive/My Drive/Colab Notebooks/Rootee


Using TensorFlow backend.


In [3]:
label_test_file= '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/testsetusedfinal.csv'
test_images_dir= '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/'
train_images_dir= '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/train/'
#train_images_dir= '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/train2/'
#label_train_file= '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/trainsample.csv'
label_train_file= '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/Label_EyeQ_train.csv'
label_valid_file= '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/validation.csv'
valid_images_dir= '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/validationnew/'

train_filenames= pd.read_csv(label_train_file)['image'].tolist()
train_labels= pd.read_csv(label_train_file)['quality'].tolist()
valid_filenames= pd.read_csv(label_valid_file)['image'].tolist()
valid_labels= pd.read_csv(label_valid_file)['quality'].tolist()
test_filenames= pd.read_csv(label_test_file)['image'].tolist()
test_labels= pd.read_csv(label_test_file)['quality'].tolist()

train_labels=  tf.keras.utils.to_categorical(train_labels, 3)
valid_labels=  tf.keras.utils.to_categorical(valid_labels, 3)
test_labels=  tf.keras.utils.to_categorical(test_labels, 3)

num_train = len(train_filenames)
num_val = len(valid_filenames)

new_train_filenames=[]
for filen in train_filenames:
  a= filen[:-4]+'png'
  a=os.path.join('/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/train', a)
  #a='/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/train/' +a
  new_train_filenames.append(a)

new_valid_filenames=[]
for filen in valid_filenames:
  a= filen[:-4]+'png'
  a='/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/validationnew/'+a
  new_valid_filenames.append(a)

new_test_filenames=[]
for filen in test_filenames:
  a= filen[:-4]+'png'
  a='/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/'+a
  new_test_filenames.append(a)

print(new_test_filenames)

['/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16276_left.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16276_right.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16285_right.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16292_left.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16292_right.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16297_right.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16298_left.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16311_left.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16311_right.png', '/content/drive/My Drive/Colab Notebooks/Rootee/EyeQ/MCF_Net/test/testsetused/16312_left.png', '/content/drive/My Drive/Colab Notebooks/Roo

In [4]:
IMAGE_SIZE = 224

def _parse_fn(filename, label):
  image_string = tf.io.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string,3)
  image_normalized = (tf.cast(image_decoded, tf.float32))#-[0.485, 0.456, 0.406])/[0.229, 0.224, 0.225]#/127.5) - 1
  image_normalized= tf.image.per_image_standardization(image_normalized)
  image_resized = tf.image.resize(image_normalized, (IMAGE_SIZE, IMAGE_SIZE))
  return image_resized, label

def _parse_fn2(filename, label):
  image_string = tf.io.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string)
  image_normalized = (tf.cast(image_decoded, tf.float32))#-[0.485, 0.456, 0.406])/[0.229, 0.224, 0.225]#/127.5) - 1
  image_normalized= tf.image.per_image_standardization(image_normalized)
  image = tf.image.resize(image_normalized, (IMAGE_SIZE, IMAGE_SIZE))
  image=tf.image.random_flip_left_right(image)
  image=tf.image.flip_up_down(image)
  
  #image=tf.keras.preprocessing.image.random_rotation(image,180)

  return image, label
'''
a,_ =_parse_fn2(new_train_filenames[1], 0)
plt.imshow(a)
plt.show()'''

'\na,_ =_parse_fn2(new_train_filenames[1], 0)\nplt.imshow(a)\nplt.show()'

In [5]:
BATCH_SIZE = 32

val_data = tf.data.Dataset.from_tensor_slices((tf.constant(new_valid_filenames), tf.constant(valid_labels)))
val_data = val_data.map(_parse_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

train_data = tf.data.Dataset.from_tensor_slices((tf.constant(new_train_filenames), tf.constant(train_labels)))
'''for i in train_data:
      print(i)'''
train_data = train_data.shuffle(buffer_size=12542).map(_parse_fn2,num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.DenseNet121(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights=None)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
sess = tf.Session()
K.set_session(sess)

learning_rate = 0.001
maxpool_layer = tf.keras.layers.GlobalMaxPooling2D()
prediction_layer = tf.keras.layers.Dense(3, activation='sigmoid', name="outputLayer")

model = tf.keras.Sequential([
    base_model,
    maxpool_layer,
    prediction_layer
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_max_pooling2d (Global (None, 1024)              0         
_________________________________________________________________
outputLayer (Dense)          (None, 3)                 3075      
Total params: 7,040,579
Trainable params: 6,956,931
Non-trainable params: 83,648
_________________________________________________________________


In [7]:
from tensorflow.keras.models import load_model
num_epochs = 1
steps_per_epoch = round(num_train)//BATCH_SIZE
val_steps = round(num_val)//BATCH_SIZE
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
							  factor = 0.1,
                mode = 'min',
							  patience = 3,
							  verbose = 1)

In [39]:
from tensorflow.keras.models import load_model

checkpoint_path= 'try9.h5'
model=load_model('try9.h5')
#initial_epoch = model.optimizer.iterations // steps_per_epoch
#print(initial_epoch)

In [8]:
#filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
#checkpoint = ModelCheckpoint('try19.h5', monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = checkpoint
initial_epoch = 0
#model.load_weights(checkpoint_filepath)

history = model.fit(train_data,
                    epochs=num_epochs,
                    steps_per_epoch = steps_per_epoch,
                    validation_data=val_data, 
                    validation_steps=val_steps,
                    verbose=1,
                    callbacks=[reduce_lr],
                    initial_epoch=initial_epoch,
                    use_multiprocessing=True,
                    workers=4)#.history
builder = tf.saved_model.builder.SavedModelBuilder("mymodel3")
# Tag the model, required for Go
builder.add_meta_graph_and_variables(sess, ["serve"])
builder.save()
sess.close()
#model.save_weights('try9.h5')
print("here")
#model.save('try9save.h5')
#model.save("try9savefolde")

Train on 391 steps, validate on 68 steps
391/391 [==============================] - 2624s 7s/step - loss: 0.3406 - acc: 0.8702 - val_loss: 0.4971 - val_acc: 0.7927
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: mymodel3/saved_model.pb
here


In [ ]:
test_data = tf.data.Dataset.from_tensor_slices((tf.constant(new_test_filenames), tf.constant(test_labels)))
test_data = test_data.map(_parse_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
score, acc = model.evaluate(test_data)
print(acc)

46/46 [==============================] - 477s 10s/step - loss: 0.1596 - accuracy: 0.9104
0.910442590713501


In [1]:
filename='poorimage.jpg'
image_string = tf.io.read_file(filename)
image_decoded = tf.image.decode_jpeg(image_string,3)
image_normalized = (tf.cast(image_decoded, tf.float32))#-[0.485, 0.456, 0.406])/[0.229, 0.224, 0.225]#/127.5) - 1
image_normalized= tf.image.per_image_standardization(image_normalized)
image_resized = tf.image.resize(image_normalized, (IMAGE_SIZE, IMAGE_SIZE))
image_resized = tf.expand_dims(image_resized,0)
print(image_resized.shape)

model.predict(image_resized)

NameError: ignored

In [3]:
from tensorflow.keras.models import load_model
model = load_model('try9savefolde')
print(model.outputs)
# [<tf.Tensor 'dense_1/Identity:0' shape=(None, 3) dtype=float32>]
print(model.inputs)

print(out.op.name for out in model.outputs)

[<tf.Tensor 'dense_1/Identity:0' shape=(None, 3) dtype=float32>]
[<tf.Tensor 'densenet121_input:0' shape=(None, 224, 224, 3) dtype=float32>]
<generator object <genexpr> at 0x7fa89d809e08>


In [10]:
import tensorflow as tf

GRAPH_PB_PATH = '/content/drive/My Drive/Colab Notebooks/Rootee/mymodel3/saved_model.pb'
with tf.Session() as sess:
   print("load graph")
   with tf.io.gfile.GFile(GRAPH_PB_PATH,'rb') as f:
       graph_def = tf.GraphDef()
   graph_def.ParseFromString(f.read())
   sess.graph.as_default()
   tf.import_graph_def(graph_def, name='')
   graph_nodes=[n for n in graph_def.node]
   names = []
   for t in graph_nodes:
      names.append(t.name)
   print(names)

load graph


DecodeError: ignored

In [3]:
!pip uninstall tensorflow
!pip install tensorflow==1.15

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 412.3MB 44kB/s 
     |████████████████████████████████| 3.8MB 33.2MB/s 
     |████████████████████████████████| 512kB 36.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=fcb9c52998aad7ab7d5b1b3f26fea5cf9a076bf49e77b6a262e932bf92f4350a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have 

In [5]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


INFO:tensorflow:Froze 1588 variables.


INFO:tensorflow:Froze 1588 variables.


INFO:tensorflow:Converted 1588 variables to const ops.


INFO:tensorflow:Converted 1588 variables to const ops.


In [6]:
# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, "model", "retina.pb", as_text=False)

'model/retina.pb'

In [5]:
import tensorflow as tf
from tensorflow.python.platform import gfile

def load_graph(frozen_graph_filename):
    with tf.io.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        
        graph_def.ParseFromString(f.read())
        
    with tf.Graph().as_default() as graph:
     
        tf.import_graph_def(graph_def)
    return graph

def analyze_inputs_outputs(graph):
    ops = graph.get_operations()
    outputs_set = set(ops)
    inputs = []
    for op in ops:
        if len(op.inputs) == 0 and op.type != 'Const':
            inputs.append(op)
        else:
            for input_tensor in op.inputs:
                if input_tensor.op in outputs_set:
                    outputs_set.remove(input_tensor.op)
    outputs = list(outputs_set)
    return (inputs, outputs)
path='/content/drive/My Drive/Colab Notebooks/Rootee/try9savefolde'
graphs=load_graph(path)
inputs,outputs= analyze_inputs_outputs(graphs)
print(inputs)
print(outputs)

[]
[]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  
